In [45]:
import os
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.utilities.dataframe_functions import impute
from tqdm import tqdm

In [46]:

def find_subsegments_indices(values):
    """
    A function used to find uninterrupted segments in a series of values and
    returns an array where each row contains the start and end index of samples
    belonging to one uninterrupted segment.

    Parameters
    ----------
    values : numpy.array
        A 1-dimensional numpy array of values.

    Returns
    -------
    segments : numpy.ndarray
        A 2-dimensional numpy array where each row contains the start and end
        index of samples belonging to one uninterrupted segment.

    """

    # If input is pd.Series transform it to numpy array
    if isinstance(values, pd.Series):
        values = values.values
    if len(values) == 0:
        raise ValueError("Segments cannot be extracted from an empty list")

    changes = np.flatnonzero(values[:-1] != values[1:]) + 1
    segments = np.column_stack(
        (
            np.concatenate(([0], changes)),
            np.concatenate((changes, [len(values)])),
        )
    )
    return segments


def get_segment_label(data, segments, label_column):
    """
    A function used to find uninterrupted segments in a series of values and
    returns an array where each row contains the start and end index of samples
    belonging to one uninterrupted segment.

    Parameters
    ----------
    data : pandas.DataFrame
        A dataframe from which the label will be extracted,
        based on the provided segments
    segments : numpy.ndarray
        A 2-dimensional numpy array where each row contains the start and end
        index of samples belonging to one uninterrupted segment.
    label_column : str
        The name of the label column

    Returns
    -------
    labels : list
        A list of labels corresponding to the number of segments given

    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input data type is not appropriate.")
    try:
        iter(segments)
    except TypeError:
        raise ValueError('Provided values for "segments" is not iterable')
    if label_column not in data.columns:
        raise ValueError("Target column not found in DataFrame")

    labels = []
    for segment in segments:
        sel_label = data.iloc[
            segment[0]: segment[1], data.columns.get_loc(label_column)
        ]
        if sel_label.shape[0] != 0:
            labels.append(sel_label.unique()[0])
    return labels


def find_label_segments(data, label_column):
    """
    A function used to find uninterrupted segments in a dataframe and returns
    a new dataframe where each row contains the start index, end index,
    and the label belonging to one uninterrupted segment.

    Parameters
    ----------
    data : pandas.DataFrame
        A dataframe from which the segments will be extracted
    label_column : str
        The name of the column from which subsegments will be created

    Returns
    -------
    res : pandas.DataFrame
        A pandas dataframe where each row contains the start index, end index,
        and the label belonging to one uninterrupted segment.

    """
    if not isinstance(data, pd.DataFrame):
        raise ValueError("Input type is not appropriate.")
    if label_column not in data.columns:
        raise ValueError("Target column not found in DataFrame")

    segments = find_subsegments_indices(data[label_column])
    labels = get_segment_label(data, segments, label_column)
    if len(labels) != len(segments):
        raise ValueError(
            f"Length mismatch between segments {(len(segments))}"
            + f" and labels {(len(labels))}"
        )
    segments = pd.DataFrame(segments, columns=["start", "end"])
    segments["duration"] = segments["end"] - segments["start"]
    segments = segments.astype("int")
    segments["Label"] = labels
    return segments

def assign_segment_ids(segments_df):
    segments_df['Segment_ID'] = range(1, len(segments_df) + 1)
    return segments_df

def extract_eating_segments(data, label_column='Label'):
    segments_df = find_label_segments(data, label_column)
    eating_segments_df = segments_df[segments_df['Label'] == 1].copy()
    eating_segments_df = assign_segment_ids(eating_segments_df)
    return eating_segments_df

def create_eating_segment_dataframe(data, segments_df):
    eating_data = pd.DataFrame()
    for _, row in segments_df.iterrows():
        start = int(row['start'])
        end = int(row['end'])
        segment = data.iloc[start:end].copy()
        segment['Segment_ID'] = row['Segment_ID']
        eating_data = pd.concat([eating_data, segment], ignore_index=True)
    return eating_data

In [47]:
custom_settings = {
    "mean": None,
    "standard_deviation": None,
    "maximum": None,
    "minimum": None,
    "skewness": None,
    "kurtosis": None,
}
data_path = r"C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data"
output_path = r"C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1"
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Initializing dictionaries to store data and eating segments for each person
person_data = {}
eating_segments = {}
features_dict= {}
labels_dict = {}

# Traversing through each CSV file
for item in os.listdir(data_path):
    item_path = os.path.join(data_path, item)  # Path to each person's folder

    if os.path.isdir(item_path):
        files = os.listdir(item_path)
        person_data[item] = {}  # Dictionary to store data for each person
        eating_segments[item] = {}  # Dictionary to store eating segments for each person
        features_dict[item] = [] # List to store extracted features for each person
        labels_dict[item] = [] # List to store labels for each person

        # Initializing a list to store dataframes for each person
        dfs = []

        # Traversing through each file in the person's folder
        for file in files:
            file_path = os.path.join(item_path, file)

            # Reading CSV file
            if file.endswith(".csv"):
                df = pd.read_csv(file_path)
                print(f"Read {file_path} with shape {df.shape}")
                dfs.append(df)  # Append dataframe to list

        # Concatenate dataframes into one combined dataframe
        if dfs:  # Check if list is not empty
            combined_data = pd.concat(dfs, ignore_index=True)
            print(f"Combined data shape for {item}: {combined_data.shape}")

            required_columns = ['Label', 'Label1']
            for col in required_columns:
                if col not in combined_data.columns:
                    raise ValueError(f"Column {col} not found in the data for {item}")


            eating_segments_df = extract_eating_segments(combined_data, label_column='Label')
            eating_data = create_eating_segment_dataframe(combined_data, eating_segments_df)

            # Perform feature extraction with progress bar
            extracted_features = extract_features(
                eating_data,
                column_id="Segment_ID",
                default_fc_parameters=custom_settings,
                disable_progressbar=False  # Show the progress bar
            )

            # Impute missing values
            impute(extracted_features)

            features_dict[item] = extracted_features

            # Save extracted features to a CSV file
            extracted_features.to_csv(os.path.join(output_path, f"{item}_extracted_features.csv"), index=False)
            print(f"Extracted features for {item} saved to {os.path.join(output_path, f'{item}_extracted_features.csv')}")

# Output features_dict to examine the extracted features for each person
print(features_dict)

Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Almond left.csv with shape (2893, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Almond right.csv with shape (2425, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Banana left.csv with shape (1617, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Banana right.csv with shape (1788, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Carrot rate.csv with shape (11416, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Cereal bar left.csv with shape (2992, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP1\Cereal bar right.csv with shape (2777, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTE

Feature Extraction: 100%|██████████| 10/10 [00:06<00:00,  1.63it/s]


Extracted features for EMTEQRAP1 saved to C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1\EMTEQRAP1_extracted_features.csv
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Almond left.csv with shape (2996, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Almond right.csv with shape (2872, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Banana left.csv with shape (2376, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Banana right.csv with shape (2212, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Cereal bar left.csv with shape (3274, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP10\Cereal bar right.csv with shape (3250, 34)
Read C:\Users\Mia\Desktop\Информациски сист

Feature Extraction: 100%|██████████| 10/10 [00:03<00:00,  3.01it/s]


Extracted features for EMTEQRAP10 saved to C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1\EMTEQRAP10_extracted_features.csv
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP11\Almond left.csv with shape (4610, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP11\Banana right.csv with shape (2318, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP11\Cereal bar right.csv with shape (3198, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQRAP11\Eating.csv with shape (40051, 34)
Combined data shape for EMTEQRAP11: (50177, 34)


...

Extracted features for EMTEQTHP7 saved to C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1\EMTEQTHP7_extracted_features.csv
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Almond left.csv with shape (4402, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Almond right.csv with shape (4486, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Banana left.csv with shape (2391, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Banana right.csv with shape (2817, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Carrot rate.csv with shape (13224, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP8\Cereal bar left.csv with shape (2997, 34)
Read C:\Users\Mia\Desktop\Информациски системи и голе

Feature Extraction: 100%|██████████| 10/10 [00:03<00:00,  2.72it/s]


Extracted features for EMTEQTHP8 saved to C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1\EMTEQTHP8_extracted_features.csv
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Almond left.csv with shape (3761, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Almond right.csv with shape (3113, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Banana left.csv with shape (1304, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Banana right.csv with shape (1281, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Carrot rate.csv with shape (12591, 34)
Read C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data\EMTEQTHP9\Cereal bar left.csv with shape (2742, 34)
Read C:\Users\Mia\Desktop\Информациски системи и голе

Feature Extraction: 100%|██████████| 10/10 [00:04<00:00,  2.47it/s]


Extracted features for EMTEQTHP9 saved to C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\extracted_features1\EMTEQTHP9_extracted_features.csv
{'.ipynb_checkpoints': [], 'EMTEQRAP1':       Timestamp__mean  Timestamp__standard_deviation  Timestamp__maximum  \
1.0      1.707496e+09                       4.255074        1.707496e+09   
2.0      1.707496e+09                       5.057589        1.707496e+09   
3.0      1.707496e+09                       4.012580        1.707496e+09   
4.0      1.707496e+09                       4.023783        1.707496e+09   
5.0      1.707496e+09                       3.764319        1.707496e+09   
6.0      1.707496e+09                       3.521832        1.707496e+09   
7.0      1.707496e+09                       1.974533        1.707496e+09   
8.0      1.707496e+09                       2.118868        1.707496e+09   
9.0      1.707496e+09                       2.217018        1.707496e+09   
10.0     1.707496e+09             

In [16]:
data_path = r"C:\Users\Mia\Desktop\Информациски системи и големи податоци\Seminarska\data"

# Initializing dictionaries to store data and eating segments for each person
person_data = {}
eating_segments = {}
filtered_segments_dict = {}  # Initializing dictionary to store filtered segments for each person (only eating segments)

# Traversing through each CSV file
for item in os.listdir(data_path):
    item_path = os.path.join(data_path, item)  # Path to each person's folder

    if os.path.isdir(item_path):
        files = os.listdir(item_path)
        person_data[item] = {}  # Dictionary to store data for each person
        eating_segments[item] = {}  # Dictionary to store eating segments for each person
        filtered_segments_dict[item] = {} # Dictionary to store label = 1.0 segments for each person

        # Initialize list to store dataframes for each person
        dfs = []

        # Traversing through each file in the person's folder
        for file in files:
            file_path = os.path.join(item_path, file)

            # Reading CSV file
            if file.endswith(".csv"):
                df = pd.read_csv(file_path)
                dfs.append(df)  # Append dataframe to list

        # Concatenate dataframes into one combined dataframe
        if dfs:  # Check if list is not empty
            combined_data = pd.concat(dfs, ignore_index=True)

            person_data[item]['combined_data'] = combined_data

            eating_segments[item]['combined_data'] = find_label_segments(combined_data, 'Label')

            # Filter and store segments for each person
            for key, segment_df in eating_segments[item].items():
                # Filtering segments where label is 1.0
                filtered_segments = segment_df[segment_df['Label'] == 1.0]
                # Storing filtered segments in the dictionary
                filtered_segments_dict[item][key] = filtered_segments


# Display filtered segments for each person
for person, filtered_segments in filtered_segments_dict.items():
    print(f"Filtered segments for {person}:")
    for key, segment_df in filtered_segments.items():
        print(f"Source: {key}")
        print(segment_df)
        print()


Filtered segments for .ipynb_checkpoints:
Filtered segments for EMTEQAPP1:
Source: combined_data
     start    end  duration  Label
1      134    736       602    1.0
3      778   1316       538    1.0
5     1353   1933       580    1.0
7     2152   2682       530    1.0
9     2733   3393       660    1.0
11    3465   4116       651    1.0
13    4356   4643       287    1.0
15    4785   5222       437    1.0
17    5267   5665       398    1.0
19    6188   7642      1454    1.0
21    7700  10046      2346    1.0
25   10258  10710       452    1.0
27   11284  12941      1657    1.0
29   12983  15929      2946    1.0
33   16147  16577       430    1.0
35   17682  18305       623    1.0
37   18372  19110       738    1.0
39   19162  19694       532    1.0
41   19887  20347       460    1.0
43   20397  21018       621    1.0
45   21074  21763       689    1.0
47   24563  25077       514    1.0
49   25127  25683       556    1.0
51   26848  27643       795    1.0
53   28301  28869       568 